## 5.4 Custom Layers
### 5.4.1 Layers without Parameters

In [2]:
from mxnet import gluon,np,npx
from mxnet.gluon import nn
npx.set_np()

In [3]:
class CenteredLayer(nn.Block):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    def forward(self,X):
        return X-X.mean()
layer=CenteredLayer()
layer(np.array([1,2,3,4,5]))        

array([-2., -1.,  0.,  1.,  2.])

In [5]:
net=nn.Sequential()
net.add(nn.Dense(128),CenteredLayer())
net.initialize()

# Note: 被「居中」层处理过的数据的均值接近 0
net(np.random.uniform(size=(4,8))).mean()

array(-6.2573235e-10)

In [6]:
# 5.4.2 Layers with Parameters
class MyDense(nn.Block):
    def __init__(self, units, in_units, **kwargs):
        super().__init__(**kwargs)
        # 初始化参数
        self.weight=self.params.get('weight', shape=(in_units,units))
        self.bias=self.params.get('bias',shape=(units,))
    def forward(self,x):
        linear=np.dot(x, self.weight.data(ctx=x.ctx))+self.bias.data(ctx=x.ctx)
        return npx.relu(linear)
dense=MyDense(units=3,in_units=5)
dense.collect_params()

mydense0_ (
  Parameter mydense0_weight (shape=(5, 3), dtype=<class 'numpy.float32'>)
  Parameter mydense0_bias (shape=(3,), dtype=<class 'numpy.float32'>)
)

In [7]:
dense.initialize()
dense(np.random.uniform(size=(2,5)))

array([[0.        , 0.        , 0.00865269],
       [0.        , 0.02834726, 0.09109452]])

In [9]:
# 使用自定义层
net=nn.Sequential()
net.add(MyDense(units=8,in_units=64),
        MyDense(units=1,in_units=8))
net.initialize()
net(np.random.uniform(size=(2,64)))

array([[0.03710656],
       [0.04621164]])